<a href="https://colab.research.google.com/github/tulee3474/AI_Resume_Builder/blob/main/job_description_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install spacy-transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [6]:
!git clone https://github.com/tulee3474/AI_Resume_Builder

Cloning into 'AI_Resume_Builder'...
remote: Enumerating objects: 423, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 423 (delta 3), reused 10 (delta 2), pack-reused 412
Receiving objects: 100% (423/423), 1.87 MiB | 16.35 MiB/s, done.
Resolving deltas: 100% (90/90), done.


In [7]:
%cd /content/AI_Resume_Builder/
!git pull

/content/AI_Resume_Builder
Already up to date.


In [9]:
import json
cv_data=json.load(open('/content/AI_Resume_Builder/Job_description_annotated/mergerd jd.json','r'))

In [10]:
!python -m spacy init fill-config /content/AI_Resume_Builder/base_config.cfg /content/AI_Resume_Builder/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/AI_Resume_Builder/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [11]:
def get_spacy_doc(file, data):
  nlp=spacy.blank('en')
  db=DocBin()

  for item in data:
    text = item['text']
    doc=nlp.make_doc(text)
    annot=item['label']

    
    ents=[]
    entity_indices=[]

    for start, end, label in annot:
      skip_entity=False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
         continue
        
      entity_indices = entity_indices + list(range(start, end))

      try:
        span=doc.char_span(start,end,label=label,alignment_mode='contract')
      except:
        continue
        
      if span is None:
        err_data=str([start,end]) + "     " + str(text) + "\n"
        file.write(err_data)
        
      else:
        ents.append(span)

    try:
        doc.ents=ents
        db.add(doc)
    except:
      pass


  return db

In [13]:
from sklearn.model_selection import train_test_split
train, test=train_test_split(cv_data,test_size=0.3)

In [14]:
len(train), len(test)

(111, 48)

In [15]:
cv_data[0]

{'id': 19,
 'label': [[0, 5, 'company'],
  [337, 357, 'project'],
  [362, 418, 'project'],
  [421, 459, 'job'],
  [475, 481, 'required skills'],
  [483, 488, 'required skills'],
  [489, 500, 'required skills'],
  [505, 513, 'required skills'],
  [642, 690, 'project'],
  [695, 772, 'project'],
  [809, 847, 'benefit'],
  [887, 927, 'benefit'],
  [989, 1057, 'benefit'],
  [1160, 1218, 'benefit'],
  [1464, 1532, 'benefit']],
 'text': "Yapz is a cutting edge, growth-stage startup that allows you to find, connect, and co-host events with clubs in your local area. In addition to their schooling, social, family and work lives, college students partake in extracurricular clubs on campus. To allocate enough time to make those clubs successful is extremely difficult. Yapz makes running a club and finding partner clubs for events easy and time efficient.\n\nWe are looking for a Backend Developer who works with Python, flask, SQLAlchemy and Postgres. The developer should also have a practical under

In [16]:
file=open('error.txt','w')
db=get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

In [17]:
db.tokens

[array([[15359919646060204151,  2672670766848947069,                    0,
         ...,                    0,                    0,
                            0],
        [18072887903199230677, 18072887903199230677,                    0,
         ...,                    0,                    0,
                            0],
        [12363111293408059666, 12363111293408059666,                    0,
         ...,                    0,                    0,
                            0],
        ...,
        [12870980204702107685,  7296015427201163752,                    0,
         ...,                    0,                    0,
                            0],
        [15884554869126768810, 15884554869126768810,                    0,
         ...,                    0,                    0,
                            0],
        [  962983613142996970,   962983613142996970,                    0,
         ...,                    0,                    0,
                            0

In [18]:
!python -m spacy train /content/AI_Resume_Builder/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-09 22:31:09,672] [INFO] Set up nlp object from config
[2022-08-09 22:31:09,684] [INFO] Pipeline: ['transformer', 'ner']
[2022-08-09 22:31:09,688] [INFO] Created vocabulary
[2022-08-09 22:31:09,690] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 458kB/s]
Downloading: 100% 878k/878k [00:00<00:00, 9.68MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 4.80MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 10.4MB/s]
Downloading: 100% 478M/478M [00:08<00:00, 62.3MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a mod

In [19]:
nlp=spacy.load('/content/AI_Resume_Builder/output/model-best')

In [20]:
doc=nlp('')
for ent in doc.ents:
  print(ent.text, "    ->>>> ", ent.label_)

In [21]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 4.0 MB/s 


In [22]:
import sys, fitz

In [23]:
for x in range(1,11):
  fname='/content/AI_Resume_Builder/jd test/'+str(x)+'.pdf'
  doc=fitz.open(fname)
  print('job description num '+str(x)+' test')
  text=" "
  for page in doc:
    text=text+str(page.get_text())
  text=text.strip()
  text=' '.join(text.split())
  doc=nlp(text)
  for ent in doc.ents:
    print(ent.text, " ->>>>>> ", ent.label_)
  print('\n')

job description num 1 test
Django Developer  ->>>>>>  job
Self managing  ->>>>>>  required skills
1+ years  ->>>>>>  required year
Python software development  ->>>>>>  required skills
2+ years  ->>>>>>  required year
System Integration using XML/REST  ->>>>>>  required skills
Web Frameworks  ->>>>>>  required skills
GIT  ->>>>>>  preferred skills
Frontend skills  ->>>>>>  required skills
HTML  ->>>>>>  required skills
CSS  ->>>>>>  required skills
Javascript  ->>>>>>  required skills
JQuery  ->>>>>>  required skills
B.Tech/B.E.  ->>>>>>  preferred skills
Computers  ->>>>>>  preferred skills
Electrical, Electronics/Telecommunication  ->>>>>>  preferred skills
B.Sc  ->>>>>>  preferred skills
- Computers  ->>>>>>  preferred skills


job description num 2 test
DevOps Engineer  ->>>>>>  job
Kubernetes  ->>>>>>  preferred skills
3-5 yrs min)  ->>>>>>  required year
3-5 yrs min  ->>>>>>  required year
AWS  ->>>>>>  required skills
3-5 Yrs  ->>>>>>  required year
3-5 yrs  ->>>>>>  required ye

In [24]:
!zip -r /content/AI_Resume_Builder/job_best.zip /content/AI_Resume_Builder/output/model-best

  adding: content/AI_Resume_Builder/output/model-best/ (stored 0%)
  adding: content/AI_Resume_Builder/output/model-best/ner/ (stored 0%)
  adding: content/AI_Resume_Builder/output/model-best/ner/moves (deflated 77%)
  adding: content/AI_Resume_Builder/output/model-best/ner/model (deflated 8%)
  adding: content/AI_Resume_Builder/output/model-best/ner/cfg (deflated 33%)
  adding: content/AI_Resume_Builder/output/model-best/tokenizer (deflated 81%)
  adding: content/AI_Resume_Builder/output/model-best/config.cfg (deflated 61%)
  adding: content/AI_Resume_Builder/output/model-best/transformer/ (stored 0%)
  adding: content/AI_Resume_Builder/output/model-best/transformer/model (deflated 14%)
  adding: content/AI_Resume_Builder/output/model-best/transformer/cfg (stored 0%)
  adding: content/AI_Resume_Builder/output/model-best/vocab/ (stored 0%)
  adding: content/AI_Resume_Builder/output/model-best/vocab/strings.json (deflated 73%)
  adding: content/AI_Resume_Builder/output/model-best/vocab/

In [25]:
from google.colab import files
files.download("/content/AI_Resume_Builder/job_best.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>